In [35]:
import json
from pathlib import Path
from datetime import datetime

import keras
from keras import Sequential, Input
import keras_tuner
from keras.callbacks import EarlyStopping
from keras.layers import LSTM, Dropout, Dense
from keras.metrics import RootMeanSquaredError
from keras.src.optimizers import Adam

from utils.centralized_training import _build_centralized_model_dataset
from gossiplearning.config import Config

from azurefunctions_utils import load_dataset

In [36]:
with open("azurefunctions_config.json", "r") as f:
  config = Config.model_validate(json.load(f))

INPUT_SHAPE = (
  config.training.input_timesteps, config.training.n_input_features
)

max_n_trials = 100
max_n_epochs = 100

In [37]:
def model_builder(hp: keras_tuner.HyperParameters) -> keras.Model:
    model = Sequential()
    # LSTM layer
    model.add(
        LSTM(
            hp.Int(
                min_value=16, max_value=320, step=16, name="lstm_layer_1_dim"
            ),
            activation="tanh",
            input_shape=INPUT_SHAPE,
            return_sequences=True,
        )
    )
    # dropout
    if hp.Boolean("dropout_1"):
        model.add(
            Dropout(
                hp.Float(
                    name="dropout_1_rate", 
                    min_value=0.1, 
                    max_value=0.5, 
                    step=0.05
                )
            )
        )
    # LSTM layer
    if hp.Boolean("lstm_2"):
        # model.add(RepeatVector(1)),
        model.add(
            LSTM(
                hp.Int(
                    min_value=16, 
                    max_value=320, 
                    step=16, 
                    name="lstm_layer_2_dim"
                ),
                activation="tanh",
                input_shape=(4, 14),
                return_sequences=True,
            )
        )
    # dropout
    if hp.Boolean("dropout_2"):
        model.add(
            Dropout(
                hp.Float(
                    name="dropout_2_rate", 
                    min_value=0.1, 
                    max_value=0.5, 
                    step=0.05
                )
            )
        )
    # LSTM layer
    if hp.Boolean("lstm_3"):
        # model.add(RepeatVector(1)),
        model.add(
            LSTM(
                hp.Int(
                    min_value=16, 
                    max_value=320, 
                    step=16, 
                    name="lstm_layer_3_dim"
                ),
                activation="tanh",
                input_shape=(4, 14),
                # return_sequences=True,
            )
        )
    # dropout
    if hp.Boolean("dropout_3"):
        model.add(
            Dropout(
                hp.Float(
                    name="dropout_3_rate", 
                    min_value=0.1, 
                    max_value=0.5, 
                    step=0.05
                )
            )
        )
    # Dense layer
    if hp.Boolean("dense"):
        model.add(
            Dense(
                hp.Int(min_value=16, max_value=320, step=16, name="dense_dim"),
                activation="relu"
            )
        )
    # output layer
    model.add(
        Dense(1, activation='relu')
    )
    # optimizer
    optz = Adam(
        learning_rate=hp.Float(
            min_value=0.0005, max_value=0.01, step=0.0005, name="learning_rate"
        )
    )
    # compile model
    model.compile(
        optimizer=optz, 
        loss='mse', 
        metrics=["mae", "msle", "mse", "mape", RootMeanSquaredError()]
    )
    return model

In [38]:
tuner = keras_tuner.RandomSearch(
    model_builder,
    objective='val_loss',
    max_trials=max_n_trials,
    directory="../experiments/hyperparameter_tuning",
    project_name=datetime.now().strftime("%Y-%m-%d_%H-%M-%S.%f")
)

In [39]:
# train, val, _ = _build_centralized_model_dataset(
#   Path("data/datasets/4func_10nodes/0"), config
# )

train, val, _ = load_dataset(
  "../experiments/azurefunctions-dataset2019/10n_k3_15min/seed4850/0", 
  "centralized"
)

In [40]:
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=config.training.patience,
    min_delta=config.training.min_delta,
    restore_best_weights=True
)

tuner.search(
    train[0], 
    train[1], 
    validation_data=val, 
    callbacks=[early_stopping], 
    epochs=max_n_epochs
)

Trial 24 Complete [00h 00m 14s]
val_loss: 38090.33203125

Best val_loss So Far: 30435.40234375
Total elapsed time: 00h 15m 24s

Search: Running Trial #25

Value             |Best Value So Far |Hyperparameter
160               |16                |lstm_layer_1_dim
True              |True              |dropout_1
False             |False             |lstm_2
True              |True              |dropout_2
True              |True              |lstm_3
False             |False             |dropout_3
False             |True              |dense
0.0095            |0.001             |learning_rate
0.4               |0.1               |dropout_1_rate
0.25              |0.1               |dropout_2_rate
64                |16                |lstm_layer_3_dim
96                |16                |dense_dim
0.15              |None              |dropout_3_rate
32                |None              |lstm_layer_2_dim

Epoch 1/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 75763.9141 - mae: 166.0795 -

KeyboardInterrupt: 

In [34]:
tuner.results_summary(num_trials=1)

Results summary
Results in ../experiments/hyperparameter_tuning/2025-02-14_10-37-20.252526
Showing 1 best trials
Objective(name="val_loss", direction="min")

Trial 001 summary
Hyperparameters:
lstm_layer_1_dim: 48
dropout_1: False
lstm_2: True
dropout_2: False
lstm_3: False
dropout_3: True
dense: False
learning_rate: 0.003
dropout_1_rate: 0.5
lstm_layer_2_dim: 288
dropout_2_rate: 0.4
lstm_layer_3_dim: 160
dropout_3_rate: 0.1
dense_dim: 32
Score: 74.92184448242188
